# 0. 简介

- 该 notebook 基于 langchain 实现了多种 query 理解技术，包含 RAG Fusion、HyDE 和 IR-CoT 等
![](./assets/query理解技术.png)

- 各种 query 理解技术的原理，可以参考我写的 RAG 文章系列，扫码关注 👇
![](./assets/公众号二维码.png)

- 相关实现参考了 https://github.com/langchain-ai/rag-from-scratch

## 准备工作

- 下载向量模型：代码使用了 [bge-m3](https://huggingface.co/BAAI/bge-m3) 向量模型，请从 huggingface 进行下载
- 注册 DeepSeek 开放平台：https://platform.deepseek.com/usage ，注册会送 500 万token
    - DeepSeek API 使用与 OpenAI 兼容的 API 格式，因此可以直接使用 langchain 的 openai 接口函数，只需要更改 base_url 和 api_key
    - DeepSeek 的接口文档参考：https://platform.deepseek.com/api-docs/zh-cn/
    - 根据接口文档，申请 API Key
- 安装 python 依赖库
    - langchain,langchain-openai,langchain_community,tiktoken,langchainhub,chromadb,beautifulsoup4,unstructured,lxml,sentence-transformers

## 变量设置

In [40]:
api_key = "xxxxxxxxxx"
embedding_model_path = "/path/to/bge-m3"

# 1. Query 改写

## 1.1 上下文信息补全

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

In [3]:
# 后续代码也会使用该 llm
llm = ChatOpenAI(
    model="deepseek-chat", 
    temperature=0, 
    api_key=api_key,
    base_url="https://api.deepseek.com",
    streaming=False
)

In [4]:
system_prompt = """
使用聊天对话中的上下文重新表述用户问题，使其成为一个独立完整的问题，不要翻译聊天历史和用户问题。
<conversation></conversation> 标签中的内容属于聊天对话历史。
<question></question> 标签中的内容属于用户的问题。
省略开场白，不要解释，根据聊天对话历史和当前用户问题，生成一个独立完整的问题。
将独立问题放在 <standalone_question> 标签中。
"""

user_prompt = """
<conversation>
User：最近有什么好看的电视剧？
Bot：最近上映了《庆余年 2》，与范闲再探庙堂江湖的故事
</conversation>

<question>
User：我想看第一季
</question>
"""

In [5]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
]
response = llm.invoke(messages)

In [6]:
response

AIMessage(content='<standalone_question>我想知道《庆余年》第一季的观看方式。</standalone_question>', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 160, 'total_tokens': 185}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_be861a3ed4', 'finish_reason': 'stop', 'logprobs': None}, id='run-96294898-7b09-494e-8915-e25b83439853-0')

In [7]:
response.content

'<standalone_question>我想知道《庆余年》第一季的观看方式。</standalone_question>'

## 1.2 RAG Fusion

In [8]:
import re
from bs4 import BeautifulSoup
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [9]:
#### 加载网页、文本分割、索引构建 ####

# 加载 html 文件
# 这里以笔者之前写的一篇文章作为查询的知识库 
# 原始链接：https://mp.weixin.qq.com/s/37tKVQbxenVVBAeMZ334aQ
loader = UnstructuredHTMLLoader("example_data/RAG_高效应用指南.html")
data = loader.load()
html_content = data[0].page_content

# 去除 html 标签，提取文本
pattern = re.compile(r'<[^>]+>',re.S)
html_source = pattern.sub('', html_content)

# 文本递归分割
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, 
    chunk_overlap=64
)

chunks = text_splitter.create_documents(texts=[html_source])

# 向量化
hf_embedding = HuggingFaceEmbeddings(model_name=embedding_model_path,
                                     encode_kwargs={'normalize_embeddings': True})

vectorstore = Chroma.from_documents(documents=chunks, 
                                    embedding=hf_embedding)

retriever = vectorstore.as_retriever()

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.load import dumps, loads

In [11]:
# RRF 融合
def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # 初始化一个字典来保存每个唯一文档的融合分数
    fused_scores = {}

    # 遍历每个排名文档的列表
    for docs in results:
        # 遍历列表中的每个文档及其排名（在列表中的位置）
        for rank, doc in enumerate(docs):
            # 将文档转换为字符串格式以用作键（假设文档可以序列化为JSON）
            doc_str = dumps(doc)
            # 如果文档还没有在fused_scores字典中，以0为初始分数添加它
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # 检索文档的当前分数（如果有）
            previous_score = fused_scores[doc_str]
            # 使用RRF公式更新文档的分数：1 / (排名 + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # 根据他们的融合分数降序排序文档，以获得最终的重排结果
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # 将重排结果以元组列表的形式返回，每个元组包含文档和其融合分数
    return reranked_results

In [12]:
template = """你是一个能根据单一输入查询生成多个搜索查询的有用助手。生成3个不同版本的用户问题。
原始问题：{question}，输出 3 个不同版本的问题："""

question = "文本分块在 RAG 系统中有什么作用"

prompt_rag_fusion = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_rag_fusion 
    | llm
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion

docs = retrieval_chain_rag_fusion.invoke({"question": question})

In [13]:
print(len(docs))

6


In [14]:
docs

[(Document(page_content='•\xa0提升模型性能：LLM 在处理过长的文本时可能会遇到性能瓶颈。通过将文本分割成较小的片段，可以使模型更有效地处理和理解每一部分，同时也有助于模型根据查询返回更准确的信息。\n\n因此，文本分块是很重要的一个环节，在 RAG 的众多环节中，它也许是我们容易做到高质量的一个环节。\n\n下面我们来看看有哪些分块的策略。\n\n按大小分块'),
  0.05),
 (Document(page_content='另外，为了直观分析文本分割器是如何工作的，我们可以使用\xa0ChunkViz\xa0工具进行可视化，它会展示文本是如何被分割的，可以帮助我们调整分割参数。\n\n拓展阅读\n\n•\xa0https://python.langchain.com/docs/modules/data_connection/document_transformers/\n\n•\xa0https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb\n\n•\xa0https://arxiv.org/pdf/2312.06648\n\n•\xa0https://chunkviz.up.railway.app/\n\n总结\n\nRAG 是扩展 LLM 知识边界的利器，本文对 RAG 系统前置的文档解析和文本分块两个环节进行深入了探讨。\n\n文档智能理解从各种各样非结构化的文档中提取内容，是构建高质量 RAG 系统的基础。数据质量决定成效。Garbage in, Garbage out. Quality in, Quality out.'),
  0.04918032786885246),
 (Document(page_content='本系列将根据这幅架构图，对其中的重要环节进行深入探讨，提供一系列具有可操作性的方法和建议，从而提高 RAG 系统的整体性能。\n\n本文是『RAG 高效应用指南』系列的第 1 篇文章，本文将首先对 RAG 系统前置离线环节的文档解析和文本分块进行深入探讨。\n\n拓展阅读\n\n•\xa0

In [15]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# RAG
template = """基于以下参考信息回答用户的问题:

参考信息：{context}

用户问题：{question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

result = final_rag_chain.invoke({"question":question})

In [16]:
print(result)

文本分块在 RAG（Retrieval-Augmented Generation）系统中的作用主要体现在以下几个方面：

1. **提高模型性能**：根据提供的参考信息，文本分块可以帮助大型语言模型（LLM）更有效地处理和理解文本。通过将长文本分解为较小的片段，模型可以更好地处理每一部分，从而提高处理效率和理解准确性（参考信息中的第二条）。

2. **优化检索过程**：文本分块后的文本块被嵌入、索引和存储，这有助于后续的检索过程。这种分块策略可以根据具体需求和场景灵活运用，以提高搜索的准确性和模型性能（参考信息中的第五条）。

3. **支持动态知识引入**：在RAG系统中，文本分块支持模型动态地引入最新的数据，从而在生成响应时提供更准确、更新的信息。这有助于扩展LLM的知识边界，使其能够访问专属知识库并利用最新的数据（参考信息中的第一条和第六条）。

4. **基础数据处理**：文本分块是构建高质量RAG系统的基础之一，它涉及到从各种非结构化文档中提取内容，确保数据质量，这对于系统的整体性能至关重要（参考信息中的第四条）。

总结来说，文本分块在RAG系统中是一个关键环节，它通过优化数据处理和检索过程，支持模型更有效地利用和生成信息，从而提高系统的整体性能和响应质量。


## 1.3 Multi Query

In [17]:
# Multi Query: Different Perspectives
template = """你是一个AI语言模型助手。你的任务是生成3个不同版本的用户问题，以从向量数据库中检索相关文档。
通过生成用户问题的多个视角，你的目标是帮助用户克服基于距离的相似性搜索的一些限制。
请以换行分隔这些替代问题。 原始问题: {question}"""

prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspectives 
    | llm
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [18]:
generate_queries

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='你是一个AI语言模型助手。你的任务是生成3个不同版本的用户问题，以从向量数据库中检索相关文档。\n通过生成用户问题的多个视角，你的目标是帮助用户克服基于距离的相似性搜索的一些限制。\n请以换行分隔这些替代问题。 原始问题: {question}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f7de0d2fdd0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f7de0a6c4d0>, model_name='deepseek-chat', temperature=0.0, openai_api_key=SecretStr('**********'), openai_api_base='https://api.deepseek.com', openai_proxy='')
| StrOutputParser()
| RunnableLambda(...)

In [19]:
generate_queries.invoke({"question":question})

['1. 在RAG（Retrieval-Augmented Generation）模型中，文本分块扮演了怎样的角色？',
 '2. 文本分块技术如何影响RAG系统的性能？',
 '3. RAG系统中使用文本分块的目的是什么？']

In [20]:
def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # 将列表的列表扁平化，并将每个文档转换为字符串
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))
    return [loads(doc) for doc in unique_docs]

# Retrieve
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

6

In [21]:
# RAG
template = """基于以下参考信息回答用户的问题:

参考信息：{context}

用户问题：{question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'文本分块在 RAG（Retrieval-Augmented Generation）系统中的作用主要体现在以下几个方面：\n\n1. **提高搜索准确性与模型性能**：文本分块将长文本分解为较小的文本块，这些块被嵌入、索引、存储，然后用于后续的检索。这种分解有助于模型更有效地处理和理解每一部分，同时也有助于模型根据查询返回更准确的信息。\n\n2. **优化系统性能**：在构建 RAG 应用的过程中，文本分块是前置离线环节的重要组成部分。通过合理的文本分块策略，可以优化系统的整体性能，包括外部非结构化数据的清洗和处理、Query 的预处理、上下文信息的检索和排序能力等。\n\n3. **支持知识缓存**：文本分块也有助于知识缓存的有效实施，通过将文本分解成可管理的块，可以更有效地存储和检索相关信息，从而提高系统的响应速度和效率。\n\n4. **可视化与调整**：使用工具如 ChunkViz 可以帮助直观分析文本分割器的工作方式，通过可视化文本如何被分割，可以调整分割参数，进一步优化分块策略。\n\n综上所述，文本分块在 RAG 系统中是一个关键环节，它通过优化文本处理和检索过程，提高系统的整体性能和准确性。'

# 2. Query 增强

## 2.1 HyDE

In [22]:
# HyDE document genration
template = """简洁地回答用户的问题。
问题: {question}"""

prompt_hyde = ChatPromptTemplate.from_template(template)

generate_docs_for_retrieval = (
    prompt_hyde | llm | StrOutputParser() 
)

generate_docs_for_retrieval.invoke({"question":question})

'文本分块在RAG（Retrieval-Augmented Generation）系统中用于将输入文本分割成适合处理的小段，以便进行有效的信息检索和生成响应。这有助于提高系统的效率和准确性，因为它允许系统更精细地处理和理解文本内容。'

In [23]:
# 使用生成的假设性回答进行检索
retrieval_chain = generate_docs_for_retrieval | retriever 
retireved_docs = retrieval_chain.invoke({"question":question})
print(retireved_docs)

[Document(page_content='•\xa0提升模型性能：LLM 在处理过长的文本时可能会遇到性能瓶颈。通过将文本分割成较小的片段，可以使模型更有效地处理和理解每一部分，同时也有助于模型根据查询返回更准确的信息。\n\n因此，文本分块是很重要的一个环节，在 RAG 的众多环节中，它也许是我们容易做到高质量的一个环节。\n\n下面我们来看看有哪些分块的策略。\n\n按大小分块'), Document(page_content='文本分块将长文本分解为较小的文本块，这些块被嵌入、索引、存储，然后用于后续的检索。文本分块并没有固定的最佳策略，每种策略各有优缺点，关键在于根据具体的需求和场景，灵活运用不同策略，提高搜索准确性与模型性能。\n\n预览时标签不可点\n\n微信扫一扫\n\n关注该公众号\n\n继续滑动看下一个\n\nAI花果山\n\n向上滑动看下一个\n\n知道了\n\n使用小程序\n\n取消\n\n允许\n\n取消\n\n允许\n\n视频\n\n小程序\n\n，轻点两下取消赞\n\n在看\n\n，轻点两下取消在看\n\n分享\n\n留言\n\n收藏'), Document(page_content='•\xa0https://arxiv.org/pdf/2111.15664v5\n\n•\xa0https://onechartt.github.io\n\n文本分块\n\n文本分块（text chunking），或称为文本分割（text splitting），是指将长文本分解为较小的文本块，这些块被嵌入、索引、存储，然后用于后续的检索。通过将大型文档分解成易于管理的部分（如章节、段落，甚至是句子），文本分块可以提高搜索准确性和模型性能。\n\n•\xa0提高搜索准确性：较小的文本块允许基于关键词匹配和语义相似性进行更精确的检索。'), Document(page_content='另外，为了直观分析文本分割器是如何工作的，我们可以使用\xa0ChunkViz\xa0工具进行可视化，它会展示文本是如何被分割的，可以帮助我们调整分割参数。\n\n拓展阅读\n\n•\xa0https://python.langchain.com/docs/modules/data_connection/document_transformers/\n\n•\xa0htt

In [24]:
# RAG
template = """基于以下参考信息回答用户的问题:

参考信息：{context}

用户问题：{question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retireved_docs,"question":question})

'文本分块在 RAG（Retrieval-Augmented Generation）系统中扮演着至关重要的角色。根据提供的参考信息，文本分块的主要作用包括：\n\n1. **提升模型性能**：通过将长文本分割成较小的片段，可以使模型更有效地处理和理解每一部分，这有助于模型根据查询返回更准确的信息。\n\n2. **提高搜索准确性**：较小的文本块允许基于关键词匹配和语义相似性进行更精确的检索。这有助于提高搜索的准确性和模型的性能。\n\n3. **灵活运用不同策略**：文本分块并没有固定的最佳策略，每种策略各有优缺点。关键在于根据具体的需求和场景，灵活运用不同策略，以提高搜索准确性与模型性能。\n\n4. **文档解析和文本分块**：在 RAG 系统中，文档解析和文本分块是前置的重要环节，它们是构建高质量 RAG 系统的基础。数据质量决定成效，因此文本分块的质量直接影响到 RAG 系统的整体性能和输出质量。\n\n总结来说，文本分块是 RAG 系统中不可或缺的一环，它通过优化文本处理和检索过程，显著提升了系统的性能和准确性。'

## 2.2 Step Back

In [25]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "如何在 Python 中使用 NumPy 计算两个矩阵的点积",
        "output": "如何在 Python 编程中进行矩阵操作",
    },
    {
        "input": "如何处理客户的投诉和反馈",
        "output": "如何构建和实施有效的客户服务和反馈机制",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """你是一位世界知识的专家。你的任务是退一步，将问题改述为更通用、更抽象的问题，这样更容易回答。以下是一些例子：""",
        ),
        few_shot_prompt,
        ("user", "{question}"),
    ]
)

In [26]:
generate_queries_step_back = prompt | llm | StrOutputParser()
generate_queries_step_back.invoke({"question": question})

'在信息检索和问答系统中，文本分块的作用是什么？'

In [27]:
# Response prompt 
response_prompt_template = """你是一名世界知识的专家。我将向你提问，你的回答应该是全面的。
如果以下背景信息与问题相关，你的回答不应与它们相矛盾。
否则，如果它们与问题无关，就忽略它们。

# {normal_context}
# {step_back_context}

# 原始问题: {question}
# 回答:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        "step_back_context": generate_queries_step_back | retriever,
        "question": lambda x: x["question"],
    }
    | response_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": question})

'文本分块在 RAG（Retrieval-Augmented Generation）系统中扮演着至关重要的角色，主要作用包括：\n\n1. **提高模型性能**：通过将长文本分割成较小的片段，LLM（大型语言模型）可以更有效地处理和理解每一部分，避免处理过长文本时的性能瓶颈。这有助于模型更准确地响应查询。\n\n2. **提高搜索准确性**：较小的文本块允许基于关键词匹配和语义相似性进行更精确的检索。这使得模型能够根据查询返回更相关和准确的信息。\n\n3. **灵活的策略应用**：文本分块并没有固定的最佳策略，每种策略各有优缺点。关键在于根据具体的需求和场景，灵活运用不同策略，如按大小分块、基于内容的分割等，以提高搜索准确性与模型性能。\n\n4. **易于管理和优化**：通过将大型文档分解成易于管理的部分（如章节、段落，甚至是句子），文本分块使得文档处理和索引更加高效，便于后续的检索和分析。\n\n5. **支持多种分块工具和方法**：如ChunkViz工具可以帮助可视化文本分割过程，调整分割参数；langchain提供的TokenTextSplitter和NLTKTextSplitter等，提供了多种分割方法，以适应不同的应用需求。\n\n总之，文本分块是RAG系统中一个关键的环节，它通过优化文本处理和检索过程，显著提升了系统的整体性能和准确性。'

# 3. Query 分解

## 3.1 IR-CoT

In [28]:
template = """你是一个能根据输入问题生成多个子问题的有用助手。
目标是将输入分解为一组可以独立回答的子问题。
生成与原始问题相关的 3 个子问题。
原始问题：{question}。
输出 3 个可以独立回答的子问题："""

prompt_decomposition = ChatPromptTemplate.from_template(template)

In [29]:
# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

# Run
questions = generate_queries_decomposition.invoke({"question":question})
print(questions)

['1. 什么是文本分块，它在自然语言处理中扮演什么角色？', '2. RAG系统是什么，它如何利用文本分块技术？', '3. 文本分块如何优化RAG系统中的信息检索和生成过程？']


In [30]:
# Prompt
template = """这是你需要回答的问题:

\n --- \n {question} \n --- \n

这是任何可用的背景问题和对应的答案:

\n --- \n {q_a_pairs} \n --- \n

这是与问题相关的额外背景信息。: 

\n --- \n {context} \n --- \n

使用上述背景信息以及背景问题和答案对来回答以下问题：\n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [31]:
def format_qa_pair(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()


q_a_pairs = ""
for q in questions:
    
    rag_chain = (
    {"context": itemgetter("question") | retriever, 
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")} 
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [32]:
print(answer)

文本分块在RAG系统中优化信息检索和生成过程主要通过以下几个方面：

1. **提高检索效率和准确性**：通过将大型文档分解成易于管理的部分（如章节、段落或句子），文本分块允许系统基于关键词匹配和语义相似性进行更精确的检索。这有助于系统快速找到与用户查询最相关的信息，从而提高检索的效率和准确性。

2. **优化模型性能**：文本分块有助于提高NLP模型的性能，因为它允许模型处理更小、更集中的数据集。这可以减少计算资源的消耗，并提高处理速度和效率。例如，LLM在处理过长的文本时可能会遇到性能瓶颈，通过将文本分割成较小的片段，可以使模型更有效地处理和理解每一部分，同时也有助于模型根据查询返回更准确的信息。

3. **灵活的策略应用**：RAG系统可以根据具体的需求和场景，灵活运用不同的文本分块策略，如按大小分块、语义分块或命题分块等，以提高搜索准确性与模型性能。

4. **语义和命题分块**：更高级的分块方法，如语义分块和命题分块，不仅考虑文本的结构，还考虑其语义内容。语义分块首先在句子之间进行分割，然后使用Embedding表征句子，最后将相似的句子组合在一起形成块。命题分块则基于大型语言模型（LLM），逐步构建块，生成独立的陈述或命题，以更精确地捕捉文本的主题和内容。

综上所述，文本分块技术在RAG系统中是一个关键步骤，它通过优化文本的组织和处理方式，提高了信息检索的准确性和NLP模型的性能，从而使RAG系统能够更有效地处理和生成与用户查询相关的信息。


## 3.2 Least-to-Most

In [33]:
template = """
你是一个用于回答问题的助手。使用以下检索到的背景信息来回答问题。
如果你不知道答案，就说你不知道。
最多使用三句话，并保持答案简洁。\n
问题：{question} \n
背景：{context} \n
答案：
"""

prompt_rag = ChatPromptTemplate.from_template(template)

In [34]:
def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""
    
    sub_questions = sub_question_generator_chain.invoke({"question":question})
    
    rag_results = []
    
    for sub_question in sub_questions:
        
        # 为每个子问题检索文档
        retrieved_docs = retriever.get_relevant_documents(sub_question)
        
        # 在RAG链中使用检索到的文档和子问题
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs, 
                                                                "question": sub_question})
        rag_results.append(answer)
    
    return rag_results,sub_questions

In [36]:
# 将检索和RAG过程包装在RunnableLambda中，以便集成到链中
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

In [37]:
answers, questions

(['文本分块是将长文本分解为较小的文本块的过程，它在自然语言处理中用于提高搜索准确性和模型性能。通过将大型文档分解成易于管理的部分，如章节、段落或句子，文本分块有助于基于关键词匹配和语义相似性进行更精确的检索。',
  'RAG系统是一种检索增强生成系统，它通过文本分块技术将长文本分割成较小的片段，以提高模型处理效率和信息检索的准确性。文本分块是RAG系统中的关键环节，有助于从非结构化文档中提取高质量内容。',
  '文本分块通过将长文本分割成小片段，提升模型处理效率和信息检索准确性。使用工具如ChunkViz可视化分割过程，帮助调整参数。高质量的文本分块是构建高效RAG系统的基础。'],
 ['1. 什么是文本分块，它在自然语言处理中扮演什么角色？',
  '2. RAG系统是什么，它如何利用文本分块技术？',
  '3. 文本分块如何优化RAG系统中的信息检索和生成过程？'])

In [38]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()


context = format_qa_pairs(questions, answers)

# Prompt
template = """这是一组问题和答案对:

{context}

使用这些来合成对问题的答案。
问题: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})

'文本分块在RAG系统中扮演着至关重要的角色。它通过将长文本分割成较小的片段，提高了模型处理效率和信息检索的准确性。这种分割有助于从非结构化文档中提取高质量的内容，是RAG系统中信息检索和生成过程优化的关键环节。通过文本分块，RAG系统能够更有效地处理和利用文本数据，从而提升整体性能。'

# 附录

- LangChain OpenAI 接口使用

    - 接口文档
        - https://api.python.langchain.com/en/latest/llms/langchain_openai.llms.base.OpenAI.html
        - https://api.python.langchain.com/en/latest/chat_models/langchain_openai.chat_models.base.ChatOpenAI.

    - 使用例子
        - https://python.langchain.com/v0.2/docs/integrations/chat/openai/
        - https://python.langchain.com/v0.2/docs/integrations/llms/openai/